
# Performance And Costing Analysis of Ergo Node

## Introduction
This notebook uses `Metrics.sqlite` database collected during full node syncronization.
See `README.md` for detail of database schema.



## Setup
If necessary uncomment and run the following commands to setup necessary packages.


In [1]:
#!pip install ipython-sql
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
!jupyter nbextension enable python-markdown / main


Please specify one nbextension/package at a time


In [2]:
# open sqlite connection to perform queries
import pandas as pd
import sqlite3
conn = sqlite3.connect("../Metrics.sqlite")


## v5.0 Validation
- make sure validateTxStateful recorded for each transaction of the block (should be empty)

In [ ]:
pd.read_sql_query(f"""
    -- invalid tx_num
    select b.blockId, t.tx_count as c, b.tx_num as n
    from (select blockId, count(*) as tx_count
          from validateTxStateful
          group by blockId) as t
             join applyTransactions as b on b.blockId = t.blockId
    where c != n;
""", conn)

- make sure recored block cost = sum of recorded tx costs (should be empty)

In [ ]:
pd.read_sql_query(f"""
    -- invalid tx cost
    select b.blockId, t.sum_costs as sum_tx_costs, b.cost as block_cost
    from (select blockId, sum(cost) as sum_costs
          from validateTxStateful
          group by blockId) as t
             join applyTransactions as b on b.blockId = t.blockId
    where sum_tx_costs != block_cost;
""", conn)


- now many transactions have negative cost (should be empty)

In [ ]:
pd.read_sql_query(f"""
-- count tx with negative cost
select count(*)
from validateTxStateful
where cost < 0;
""", conn)


## v5.0 Analysis
### Block Validation Time Analysis
In this section we compare script validation time against block validation time.

#### Sizes of the recorded tables

In [ ]:
pd.read_sql_query(f"""
    select * from
    (select count(*) as appendFullBlock
    from appendFullBlock),
    (select count(*) as applyTransactions
    from applyTransactions),
    (select count(*) as createUtxoState
    from createUtxoState),
    (select count(*) as validateTxStateful
    from validateTxStateful),
    (select count(*) as verifyScript
    from verifyScript)
""", conn)


#### Comparing Stages of Block Validation
First we look at how much time of the total block validation is spent in applyTransactions.
We group and count blocks by (total time / `applyTransactions` time) ratio.
We see that `applyTransactions` is < 50% for roughly 80% of the blocks.
For more ~40% of the blocks:
1) the ratio is above 3, which means script validation is < 33%
2) further analysis shows that time is spent in creating UtxoState (after applyTransaction)
**Thus, creating UtxoState after application of transactions requires profiling and optimization.**

In [ ]:
pd.read_sql_query(f"""
select time_us / t2_us as time_ratio, count(*) as block_count
from (select b1.blockId,
             b1.height,
             b1.tx_num,
             b2.cost,
             b1.time / 1000                       as t1_us,
             b2.time / 1000                       as t2_us,
             b3.time / 1000                       as t3_us,
             (b1.time + b2.time + b3.time) / 1000 as time_us
      from appendFullBlock as b1
               join applyTransactions as b2 on b1.blockId = b2.blockId
               join createUtxoState b3 on b1.blockId = b3.blockId)
group by time_ratio
order by time_ratio;
""", conn)


#### Comparing applyTransaction of block with validateTxStateful
Here we further drill down to applyTransactions part of block validation.
Specifically, for each block we compare the time of `UtxoState.applyTransactions` with total time
of `ErgoTransaction.validateStateful` taken for all transactions in the block.
The blocks are grouped by the ratio between times.
We can see that for > 70% blocks the ration is above 2, which suggests that
**`UtxoState.applyTransactions` method need optimizations.**

In [ ]:
pd.read_sql_query(f"""
select t.time_ratio / 10 as time_ratio,
       count(*) as block_count,
       round(avg(t.block_time_us), 0)  as avg_block_time_us,
       round(avg(t.tx_time_us), 0) as avg_tx_time_us,
       round(avg(t.tx_count), 1)   as avg_tx_count
from (select b.blockId,
             tx.tx_count,
             b.time / 1000                        as block_time_us,
             tx.sum_tx_time / 1000            as tx_time_us,
             (b.time - tx.sum_tx_time) / 1000 as time_diff_us,
             b.time * 10 / tx.sum_tx_time     as time_ratio
      from (select blockId,
                   sum(time) as sum_tx_time,
                   count(*)  as tx_count
            from validateTxStateful
            group by blockId) as tx
               join applyTransactions as b on b.blockId = tx.blockId) as t
group by t.time_ratio / 10
order by t.time_ratio / 10;
""", conn)


#### Tx/Script validation time ratio
Further down to validation call stack, we observe how much `validateStateful` time larger than
script verification time.
The ratio is computed for each transaction and then the transactions are grouped by integer ratio.
Comparing times in `verifyScript` and `validateTxStateful` metrics.
We see that for most transactions, `verifyScript` is > 50% of `validateStateful`.

In [ ]:
pd.read_sql_query(f"""
select t.time_ratio / 10 as time_ratio,
       count(*) as tx_count,
       round(avg(t.tx_time_us), 1) as avg_tx_time_us,
       round(avg(t.script_time_us), 1) as avg_script_time_us,
       round(avg(t.script_count), 1) as avg_script_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
group by t.time_ratio / 10
order by t.time_ratio / 10;
""", conn)


We need to further drill down inside the mose populated group.
We build a detailed grouping of the transactions where the ratio <= 1.9,
comparing times in `verifyScript` and `validateTxStateful` metrics.

In [ ]:
pd.read_sql_query(f"""
select round(t.time_ratio * 0.1, 1) as time_ratio,
       count(*) as tx_count,
       round(avg(t.tx_time_us), 1) as avg_tx_time_us,
       round(avg(t.script_time_us), 1) as avg_script_time_us,
       round(avg(t.script_count), 1) as avg_script_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
where t.time_ratio <= 19
group by t.time_ratio
order by t.time_ratio;
""", conn)


Count transactions where script validation <= 80% of `validateStatefull`.
This DOESN'T show big potential for optimizing `validateStateful` outside script evaluation.
Comparing times in `verifyScript` and `validateTxStateful` metrics.

In [ ]:
pd.read_sql_query(f"""
select count(*) as tx_count
from (select tx.blockId,
             tx.txId,
             t.script_count,
             tx.time / 1000                   as tx_time_us,
             t.sum_script_time / 1000         as script_time_us,
             tx.time * 10 / t.sum_script_time as time_ratio
      from (select blockId,
                   txId,
                   sum(time) as sum_script_time,
                   count(*)  as script_count
            from verifyScript
            group by blockId, txId) as t
               join validateTxStateful as tx on tx.blockId = t.blockId and tx.txId = t.txId) as t
where t.time_ratio >= 12;
""", conn)


Conclusions:
- Creating UtxoState after application of transactions requires profiling and optimization.
- `UtxoState.applyTransactions` method need optimizations.
- In most of the cases the time to validate transaction (`validateStateful` method) is dominated by
the script validation time
- There is a few transactions where script validation not greater than 80% of tx validation time.
- The results suggest that `validateStateful` doesn't require optimizations, or at least it can
be done after the other parts of block validation had been optimized.

## v5.0 vs v4.0 Cross Analysis

### Cross Version validation
- Make sure recorded data for v5 and v4 have the same `height` and `tx_num` for all blockIds


In [ ]:
checks = dict(
    appendFullBlockOk=pd.read_sql_query(f"""
        -- validate appendFullBlock tables
        select *
        from appendFullBlock a1
                 join appendFullBlock4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    applyTransactionsOk=pd.read_sql_query(f"""
        select *
        from applyTransactions a1
                 join applyTransactions4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    createUtxoStateOk=pd.read_sql_query(f"""
        select *
        from createUtxoState a1
                 join createUtxoState4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0
)
if not (checks.get("appendFullBlockOk") and checks.get("applyTransactionsOk") and checks.get(
        "createUtxoStateOk")):
    ok = checks
else:
    ok = "ok"
print(checks)


### Block Validation Times
First count recorded block validations of v4 and v5 and how much of them can be
joined. This is also validation check, because `common_rows` should be equal to the minimal count.

In [ ]:
pd.read_sql_query(f"""
select *, count_rows5 - count_rows4 as v5_v4_rows_diff
from (select count(*) as count_rows5 from applyTransactions),
     (select count(*) as count_rows4 from applyTransactions4),
     (select count(*) as common_rows
      from applyTransactions as t5 join applyTransactions4 t4 on t5.blockId = t4.blockId);
""", conn)


The new v5 script interpreter is expected to perform faster comparing to v4.
The following shows the distribution of the blocks across ranges of speedup. We see that for the
most of the blocks the speedup is in a range from 1 (no speedup) to 2 (twice faster).

In [ ]:
pd.read_sql_query(f"""
select t4.time / t5.time as speedup, count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
group by speedup order by speedup;
""", conn)

We see outliers, i.e. blocks which executed slower in v5 than in v4. This can be attributed to
measurement fluctuations. Such blocks will be simply filtered out in the following analysis.

We also see a long tail of blocks with higher than 2 speedups.

In [ ]:
pd.read_sql_query(f"""
select sum(num_blocks) from (
select t4.time / t5.time as speedup, count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where speedup >= 2
group by speedup order by speedup
)
""", conn)


Let's see how the speedup is spread over ranges of the blockchain.
The block `b` falls in `range` if `range = b.height / 100000`. The blocks of each range are counted,
and the average speedup over the range is computed.

In [ ]:
pd.read_sql_query(f"""
select t5.height / 100000                            as range,
       round(avg(t4.time * 100 / t5.time * 0.01), 2) as avg_speedup,
       count(*)                                      as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where t4.time / t5.time >= 1
group by range order by range;
""", conn)


We observe that the average speedup increase towards the most recent range, which is expected, because
more and more complex contracts are used on chain over time. The v5 script interpreter is by design
faster on complex contracts.

If we further constrain speedup to be above 1.2, we get the following distribution of blocks.

In [ ]:
pd.read_sql_query(f"""
select t5.height / 100000                            as range,
       round(avg(t4.time * 100 / t5.time * 0.01), 2) as avg_speedup,
       count(*) as num_blocks
from applyTransactions t5 join applyTransactions4 t4 on t5.blockId = t4.blockId
where t4.time * 100 / t5.time * 0.01 >= 1.2
group by range order by range
""", conn)

Note, this speedups are due to improvements in script evaluation which is further analyzed in the
[next section](#script-validation-time). However, as shown in [Comparing Stages of Block
Validation](#comparing-stages-of-block-validation), for most of the blocks, script evaluation is
less than 50% of the block validation time, so the impact of script speedups is limited by this
factor.


### Script Validation Time
In this section we focus on script evaluation part only (i.e. Interpreter.verify method), which is
measured and recorded in `verifyScript` table.

First count recorded script validations of v4 and v5 and how much of them can be
joined. This is also validation check, because `common_rows` should be equal to the minimal count.

In [ ]:
pd.read_sql_query(f"""
select *, total_rows5 - total_rows4 as v5_v4_rows_diff from
(select count(*) as total_rows5
from verifyScript as s),
(select count(*) as total_rows4
from verifyScript4 as s),
(select count(*) as common_rows
from verifyScript as t5 join verifyScript4 t4
  on t5.blockId = t4.blockId and t5.txId = t4.txId and t5.boxIndex = t4.boxIndex);
""", conn)


  For common recorded script we compute the total time spend in script validation for both v4 and v5
  and compare them. Then compute the expected percent of total script time reduction after switching
  to v5.0.

In [ ]:
pd.read_sql_query(f"""
select times.total_time4 / 1000                                   as total_time4_us,
       times.total_time5 / 1000                                   as total_time5_us,
       (times.total_time4 - times.total_time5) / 1000             as total_diff_us,
       round((1 - round(times.total_time5 * 100 / times.total_time4 * 0.01, 1)) * 100, 1) as percent_of_reduction
from (select sum(t5.time) as total_time5,
             sum(t4.time) as total_time4
      from verifyScript as t5
               join verifyScript4 t4
                    on t5.blockId = t4.blockId
                        and t5.txId = t4.txId
                        and t5.boxIndex = t4.boxIndex) as times;
""", conn)


### Block Validation Cost Analysis
In this section we compare block validation costs against block validation time in order to see how
accurate cost estimation predicts the actual execution time.

The validation complexity is estimated in cost units, one cost unit corresponds approximately to
1 microsecond of execution time, thus when cost is 1000 then execution time is expected to be 1000
microseconds.

Since cost predition is a security measure, we want to be conservative and require that for most
blocks the predicted cost is larger than execution time in microseconds.

The following table counts blocks with execution time exceeding predicted cost. The blocks are
grouped by `time / cost` ratio. We see that in v4.x only 50 blocks were validated longer than
predicted. The one outlier block is measurement artefact and can be ignored.


In [ ]:
pd.read_sql_query(f"""
-- find blocks where cost is less than time_us
select min(ratio / 10) as ratio, count(*)
from (select height,
             tx_num,
             cost,
             time / 1000          as time_us,
             (time / 1000) * 10 / cost as ratio
      from applyTransactions4
      where time_us > cost)
group by ratio / 10
""", conn)



The same query with v5.0 execution data shows similar results.

In [ ]:
pd.read_sql_query(f"""
-- find blocks where cost is less than time_us
select min(ratio / 10) as ratio, count(*)
from (select height,
             tx_num,
             cost as full_cost,
             time / 1000          as time_us,
             (time / 1000) * 10 / cost as ratio
      from applyTransactions
      where time_us > full_cost)
group by ratio / 10
""", conn)


We can conclude that both v4.x and v5.0 costing can be used as the upper bound of the actual block
validation time, i.e. for most of the blocks the cost value is larger than execution time in
microseconds. How accurate this bound?

In v4.x the `cost / time` ratio is in [20 .. 70) range which is quite conservative, leaving a lot of
room for improvement.

In [ ]:
pd.read_sql_query(f"""
-- group and count blocks by cost/time ratio (v4)
select min(ratio), count(*), round(avg(tx_num), 2) as avg_tx_num
from (select height,
             tx_num,
             cost as full_cost,
             time / 1000          as time_us,
             cost / (time / 1000)  as ratio
      from applyTransactions4
      where time_us <= full_cost)
group by ratio / 10
""", conn)


Indeed, as can be seen in the following table, in v5.0 cost prediction is significantly improved so
that the `cost / time` ratio is in [1 .. 19) range for most blocks.


In [ ]:
pd.read_sql_query(f"""
-- group and count blocks by cost/time ratio (v5)
select min(ratio), count(*), round(avg(tx_num), 2) as avg_tx_num
from (select height,
             tx_num,
             cost as full_cost,
             time / 1000          as time_us,
             cost / (time / 1000)  as ratio
      from applyTransactions
      where time_us <= full_cost)
group by ratio / 10
""", conn)


The maximal cost of each block is limited by maxBlockCost parameter stored in block parameters
section of each block. This parameter can be changed by miners via voting. The following table shows
the distribution of blocks over range of cost limits.


In [ ]:
pd.read_sql_query(f"""
select min(maxCost), count(*) from applyTransactions
group by maxCost / 1000000
""", conn)


We see the cost limit significantly increased by the miners (in fact by the pool operators) from
initial 1000000 up to 7030268 current value.

Now that we have cost limits for each block, let's see how far the actual block validation costs
from that limits.


In [ ]:
pd.read_sql_query(f"""
select maxCost / cost as ratio, count(*), min(height) from applyTransactions
group by ratio
""", conn)


We see that for all blocks the actual cost is at least 2x less then cost limit. However, we also see
that for many blocks the cost is much smaller (569 times) than the limit value .
Lets investigate further the blocks with lowest and highest ratios.


In [ ]:
pd.read_sql_query(f"""
select round(maxCost * 10 / cost * 0.1, 1) as ratio, count(*), min(height) from applyTransactions
where ratio <= 3
group by ratio
""", conn)


What is the block with lowest ratio?


In [ ]:
pd.read_sql_query(f"""
select round(t5.maxCost * 10 / t5.cost * 0.1, 1) as ratio,
       t5.height, t5.tx_num, t5.maxCost,
       t5.cost as cost_v5,
       t5.time / 1000 as time_t5_us,
       7030268 / (t5.time / 1000) as scalability_v5,
       t4.cost as cost_v4,
       t4.time / 1000 as time_t4_us
from applyTransactions t5
         join applyTransactions4 t4 on t5.blockId = t4.blockId
where ratio < 2.6
""", conn)


This is the block deep in the blockchain with 114 transactions.
Note, the v4 cost hit the cost limit, so this number of transaction was maximum possible at that
time. This is a good example to illustrate the benefits of v5.0. Not only it executed the same
transaction slightly faster, but also due to more accurate costing, the predicted cost is 2.5 times
lower than v4 cost. This means the operator would have been able to include more than 300
transactions in the block thus increasing network throughput and reduce congestion.

Since then, the cost limit was increased by pool operators up to 7030268, almost 2x.
Using these numbers we can roughly estimate the number of transactions in a block as
114 * (7030268(maxCost) / 1947948(cost_v5)) = 411 transactions.

We can also estimate the potential scalability of v5.0 if we compare the actual execution time with
the current cost limit (see `scalability_v5`). With further tuning of the cost parameters (possible
via voting) the number of such transactions in one block can be 114 * 100 = 11400 (even with the
current unoptimized state management), or 11400 / 120 = 95 tx/second.

At the same time, in this case, the total block validation would take 70212 * 100 = 7021200
microseconds, which is above the recommended time limit of 5 seconds. This suggest that _pool
operators need to postpone further increasing of the maxBlockCost parameter and instead switch on to tuning
the cost parameters to make the cost prediction more accurate_.

Now, what about the other side of the maxCost/cost ratio spectrum, where cost limit is much larger
than the actual block cost.


In [ ]:
pd.read_sql_query(f"""
select round(maxCost * 10 / cost * 0.1, 1) as ratio,
       height, tx_num, maxCost, cost,
       time / 1000                         as time_us
from applyTransactions
where ratio >= 569
order by height desc
limit 20
""", conn)


We see many recent blocks (the highest `height`) with single simple transaction.

#### Conclusions
What this analysis of actual execution of Ergo Node v5.0 tell us:
- the new cost estimation is properly estimates the actual execution time of all the existing
blocks.
- the cost estimation is reasonably conservative, i.e. for 95% blocks it overestimates the actual
costs, but this overestimation is significantly lower than in v4.x
- for all blocks the estimated costs are more than 2.5 times lower than the cost limits.
- In the low boundary case shown above, the block which is the closest to the cost limit has 100+
transactions. With v5.0 this number could be higher